In [ ]:
import polars as pl
import networkx as nx
from math import sqrt
import time
import random
import rtsvg
rt = rtsvg.RACETrack()
# Create a graph
from linknode_graph_patterns import LinkNodeGraphPatterns
_patterns_ = LinkNodeGraphPatterns()
g          = _patterns_.createPattern('mesh')
# Create the dataframe version of the graph
_lu_ = {'fm':[], 'to':[]}
for edge in g.edges: _lu_['fm'].append(edge[0]), _lu_['to'].append(edge[1])
df = pl.DataFrame(_lu_)
'''
_base_network_ = '0'
_base_dir_     = '../../data/stanford/facebook/'
_layout_file_  = _base_dir_ + _base_network_ + '.layout.parquet'
_edges_ = open(_base_dir_ + _base_network_ + '.edges', 'rt').read()
_lu_ = {'fm':[], 'to':[]}
for _edge_ in _edges_.split('\n'):
    if _edge_ == '': continue
    _lu_['fm'].append(_edge_.split(' ')[0])
    _lu_['to'].append(_edge_.split(' ')[1])
df = pl.DataFrame(_lu_)
g  = rt.createNetworkXGraph(df, [('fm', 'to')])
len(df), len(set(df['fm'])), len(set(df['to'])), len(set(df['fm']) | set(df['to']))
'''
print(f'edges: {len(g.edges)} | nodes: {len(g.nodes)}')

In [ ]:
#
# "Graph Drawing by Force-directed Placement"
# Fruchterman & Reingold
# Software -- Practice and Experience, Vol. 21 (1 1), 1129-1164 (November 1991)
#
# Figure 1 (Force-directed Placement)
# - with the optimization to only consider vertices w/in 2*k of the vertex for repulsion
#
pos_history = {}
for _node_ in g.nodes: pos_history[_node_] = []

t    = 4.0
W, H = 256, 256
pos  = {}
for _node_ in g.nodes(): pos[_node_] = random.random()*W, random.random()*H
area = W*H
C   = 0.2 # paper says that C is found experimentally...
k   = C*sqrt(area/len(g.nodes()))
def f_a(z): return  z**2 / k             # pseudocode says "x" vs "z" ... paper comments say "z"
def f_r(z): return  ((k**2)/z)*(2*k - z) # updated formula when only using near neighbors for repulsion

t0 = time.time()
_iterations_ = 2*len(g.nodes())
for i in range(_iterations_):
    for _node_ in g.nodes: pos_history[_node_].append(pos[_node_])
    disp = {}
    # calculate repulsive forces
    for v in g.nodes():
        disp[v] = 0.0, 0.0
        for u in g.nodes():
            if u == v or g.has_edge(u, v): continue
            _diff_     = pos[v][0] - pos[u][0], pos[v][1] - pos[u][1]
            _diff_len_ = max(sqrt(_diff_[0]**2 + _diff_[1]**2), 0.001)
            if _diff_len_ > 2*k: continue
            disp[v]    = disp[v][0] + (_diff_[0]/_diff_len_) * f_r(_diff_len_), disp[v][1] + (_diff_[1]/_diff_len_) * f_r(_diff_len_)
    # calculate the attractive forces
    for e in g.edges():
        e_v, e_u, w = e[0], e[1], 1 if 'weight' not in g[e[0]][e[1]] else g[e[0]][e[1]]['weight']
        _diff_     = pos[e_v][0] - pos[e_u][0], pos[e_v][1] - pos[e_u][1]
        _diff_len_ = max(sqrt(_diff_[0]**2 + _diff_[1]**2), 0.001)
        disp[e_v]  = disp[e_v][0] - (_diff_[0]/(_diff_len_*w)) * f_a(_diff_len_), disp[e_v][1] - (_diff_[1]/(_diff_len_*w)) * f_a(_diff_len_)
        disp[e_u]  = disp[e_u][0] + (_diff_[0]/(_diff_len_*w)) * f_a(_diff_len_), disp[e_u][1] + (_diff_[1]/(_diff_len_*w)) * f_a(_diff_len_)
    # limit the maximum displacement to the temperature t
    # ... and then prevent from being displaced outside frame
    for v in g.nodes():
        disp_len   = max(sqrt(disp[v][0]**2 + disp[v][1]**2), 0.001)
        xmag, ymag = t if abs(disp[v][0]) > t else disp[v][0], t if abs(disp[v][1]) > t else disp[v][1]
        pos[v] = pos[v][0] + (disp[v][0]/disp_len)*xmag, pos[v][1] + (disp[v][1]/disp_len)*ymag
    t *= 0.98 # t = cool(t)

    # Rescale th positions to fit the frame
    # ... the paper has a several proposals for how to keep all the vertices in the frame
    # ... kindof unsure if this is necessary or not
    x0, y0, x1, y1 = pos[v][0], pos[v][1], pos[v][0], pos[v][1]
    for _node_ in pos.keys():
        x0, y0 = min(x0, pos[_node_][0]), min(y0, pos[_node_][1])
        x1, y1 = max(x1, pos[_node_][0]), max(y1, pos[_node_][1])
    # for _node_ in pos.keys(): pos[_node_] = W*(pos[_node_][0] - x0)/(x1-x0), H*(pos[_node_][1] - y0)/(y1-y0)

for _node_ in g.nodes: pos_history[_node_].append(pos[_node_])

x0, y0, x1, y1 = pos[v][0], pos[v][1], pos[v][0], pos[v][1]
for k in pos_history.keys():
    x0, y0 = min(x0, min(pos_history[k][0])), min(y0, min(pos_history[k][1]))
    x1, y1 = max(x1, max(pos_history[k][0])), max(y1, max(pos_history[k][1]))

t1 = time.time()

# Convert it to an animation
_dur_ = '5s'
svg = [f'<svg x="0" y="0" width="512" height="512" viewBox="{x0} {y0} {x1-x0} {y1-y0}">']
svg.append(f'<rect x="{x0}" y="{y0}"  width="{x1-x0}" height="{y1-y0}" fill="white" />')
for _edge_ in g.edges:
    u, v = _edge_[0], _edge_[1]
    svg.append(f'<line x1="{pos[u][0]}" y1="{pos[u][1]}" x2="{pos[v][0]}" y2="{pos[v][1]}" stroke="black" stroke-width="0.2">')
    svg.append(f'<animate attributeName="x1" values="{";".join([f"{_pos_[0]}" for _pos_ in pos_history[u]])}" dur="{_dur_}" repeatCount="indefinite" />')
    svg.append(f'<animate attributeName="y1" values="{";".join([f"{_pos_[1]}" for _pos_ in pos_history[u]])}" dur="{_dur_}" repeatCount="indefinite" />')
    svg.append(f'<animate attributeName="x2" values="{";".join([f"{_pos_[0]}" for _pos_ in pos_history[v]])}" dur="{_dur_}" repeatCount="indefinite" />')
    svg.append(f'<animate attributeName="y2" values="{";".join([f"{_pos_[1]}" for _pos_ in pos_history[v]])}" dur="{_dur_}" repeatCount="indefinite" />')
    svg.append('</line>')
for _node_ in g.nodes:
    svg.append(f'<circle cx="{pos_history[_node_][0][0]}" cy="{pos_history[_node_][0][1]}" r="1.2" fill="blue">')
    svg.append(f'<animate attributeName="cx" values="{";".join([f"{_pos_[0]}" for _pos_ in pos_history[_node_]])}" dur="{_dur_}" repeatCount="indefinite" />')
    svg.append(f'<animate attributeName="cy" values="{";".join([f"{_pos_[1]}" for _pos_ in pos_history[_node_]])}" dur="{_dur_}" repeatCount="indefinite" />')
    svg.append('</circle>')
svg.append('</svg>')

_lu_ = {'fm':[], 'to':[]}
for edge in g.edges: _lu_['fm'].append(edge[0]), _lu_['to'].append(edge[1])
df = pl.DataFrame(_lu_)
#rt.tile([''.join(svg), rt.link(df, [('fm','to')],pos)], spacer=10)

In [ ]:
t    = 4.0
W, H = 256, 256
area = W*H
C   = 0.2 # paper says that C is found experimentally...
k   = C*sqrt(area/len(g.nodes()))

static_nodes = None

# Distance DataFrame
_lu_    = {'fm':[], 'to':[], 't':[]}
for node in g.nodes:
    for nbor in g.neighbors(node):
        _w_ = g[node][nbor]['weight'] if 'weight' in g[node][nbor] else 1.0
        _lu_['fm'].append(node), _lu_['to'].append(nbor), _lu_['t'].append(_w_)
df_dist = pl.DataFrame(_lu_)

# Positional DataFrame
_lu_    = {'node':[], 'x':[], 'y':[], 's':[]}
for node in g.nodes: 
    _lu_['node'].append(node)
    if pos is not None and node in pos: _lu_['x'].append(pos[node][0]),      _lu_['y'].append(pos[node][1])
    else:                               _lu_['x'].append(W*random.random()), _lu_['y'].append(H*random.random()) 
    if static_nodes is not None and node in static_nodes: _lu_['s'].append(True)
    else:                                                 _lu_['s'].append(False)
df_pos  = pl.DataFrame(_lu_)

# Cross join of all of the nodes
df_cross = df_pos.drop('s')\
                 .join(df_pos.drop('s'), how='cross') \
                 .filter(pl.col('node') != pl.col('node_right'))

df_cross

In [ ]:
# Dataframe describing one off tiles
df_around = pl.DataFrame({'xi_offset':[-1,  0,  1, -1, 1, -1, 0, 1],
                          'yi_offset':[-1, -1, -1,  0, 0,  1, 1, 1]})
# Calculate the tile xi, yi based on the 2*k parameter
_df_ = df_pos.with_columns(((pl.col('x') - pl.col('x').min())//(2 * k)).cast(pl.Int32).alias('xi'),
                           ((pl.col('y') - pl.col('y').min())//(2 * k)).cast(pl.Int32).alias('yi'))
_df_ = _df_.join(df_around, how='cross')
# Determine which tiles a node needs
_df_ = _df_.with_columns((pl.col('xi_offset')+pl.col('xi')).alias('xi_join'),
                         (pl.col('yi_offset')+pl.col('yi')).alias('yi_join'))
# Join those tiles in
_df_ = _df_.join(_df_, left_on=['xi_join','yi_join'], right_on=['xi','yi']) # this removes node == node_right by default...
# Remove all the fields related to tiles
_df_ = _df_.drop(['s','xi','yi','xi_offset','yi_offset','xi_join','yi_join','s_right','xi_offset_right','yi_offset_right','xi_join_right','yi_join_right'])
# Remove edges (these are not repulsed)
_df_ = _df_.join(df_dist, left_on=['node','node_right'], right_on=['fm','to'], how='anti')
# Calculate the distance squared
_df_ = _df_.with_columns(((pl.col('x_right')-pl.col('x'))**2 + (pl.col('y_right')-pl.col('y'))**2).alias('d_squared'))
# filter out the ones that are too far
_df_ = _df_.filter(pl.col('d_squared') < (2*k)**2)
_df_